In [ ]:
import zeep
import numpy as np
import math
import time

import matplotlib.pyplot as plt

In [ ]:
client = zeep.Client('http://turbulence.pha.jhu.edu/service/turbulence.asmx?WSDL')
ArrayOfFloat = client.get_type('ns0:ArrayOfFloat')
ArrayOfArrayOfFloat = client.get_type('ns0:ArrayOfArrayOfFloat')
SpatialInterpolation = client.get_type('ns0:SpatialInterpolation')
TemporalInterpolation = client.get_type('ns0:TemporalInterpolation')

token = "uk.ac.imperial.george.dixon-dray17-29581e35"

In [ ]:
def get_fields(filter_width_n=9,origin_x=0,sample_nx=64,field="u",t=0,dataset = "isotropic1024coarse"):
    start_time = time.time()
    
    # Database Domain Size
    if dataset == "isotropic1024coarse":
        domain_nx = 1024
    elif dataset == "isotropic4096":
        domain_nx = 4096

    # Database Domain Size
    domain_x_min = 0
    domain_x_max = 2*np.pi
    domain_spacing = domain_x_max / (domain_nx)


    # Sample Domain Size
    filter_spacing = filter_width_n * domain_spacing
    print(filter_spacing)
    sample_x_min = math.ceil(filter_width_n/2) * domain_spacing + origin_x
    sample_x_max = sample_x_min + ((sample_nx-1) * filter_spacing)

    print("filter_spacing:", filter_spacing)

    x = np.linspace(sample_x_min, sample_x_max, sample_nx)
    print("x[1]-x[0]:", x[1]-x[0])
    y = x
    z = x
    X,Y,Z = np.meshgrid(x,y,z)
    X = X.ravel()
    Y = Y.ravel()
    Z = Z.ravel()

    nnp=5 #number of points
    points=np.random.rand(nnp,3)

    points = np.zeros((sample_nx**3,3))
    points[:,0] = Z
    points[:,1] = X
    points[:,2] = Y


    # convert to JHTDB structures
    x_coor=ArrayOfFloat(points[:,0].tolist())
    y_coor=ArrayOfFloat(points[:,1].tolist())
    z_coor=ArrayOfFloat(points[:,2].tolist())
    point=ArrayOfArrayOfFloat([x_coor,y_coor,z_coor]);
    
    filterwidth = filter_spacing
    spacing = filter_spacing #spacing is only used in GetBoxFilterGradient, but always provide it.

    number_of_component=9
    grad_filtered = client.service.GetFilter_Python("GetBoxFilterGradient",token,"isotropic1024coarse", field, 
                                           t, filterwidth, SpatialInterpolation("None"), point, spacing)
    grad_filtered=np.array(grad_filtered).reshape((-1, number_of_component))
    print('found gradient')
    print(np.shape(grad_filtered))
    
    number_of_component=6 # change this based on function_name, see http://turbulence.pha.jhu.edu/webquery/query.aspx
    SGSsym=client.service.GetFilter_Python("GetBoxFilterSGSsymtensor",token,"isotropic1024coarse", field, 
                                           t, filterwidth, SpatialInterpolation("None"), point, spacing) 
    SGSsym=np.array(SGSsym).reshape((-1, number_of_component))
    print('found SGS')
    print(np.shape(SGSsym))
    
#     number_of_component=9 # change this based on function_name, see http://turbulence.pha.jhu.edu/webquery/query.aspx
#     pressure_grad=client.service.GetFilter_Python("GetBoxFilterGradient",token,"isotropic1024coarse", "p", 
#                                            t, filterwidth, SpatialInterpolation("None"), point, spacing) 
#     pressure_grad=np.array(pressure_grad).reshape((-1, number_of_component))
#     print(np.shape(pressure_grad))
    
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return grad_filtered, SGSsym #, pressure_grad


In [ ]:
cube_grad, cube_SGS = get_fields(filter_width_n=9,origin_x=np.pi,sample_nx=50,field="u",t=5)

In [ ]:
# cube_pressure = cube_pressure_9[:,:3]
# input_data = np.concatenate((cube_grad,cube_pressure),axis=1)
# print(np.shape(input_data))
# np.save('cube_3_grad_p', input_data)

In [ ]:
np.save('cube_6_grad', cube_grad)

np.save('cube_6_SGS', cube_SGS)

In [ ]:
cube_grad_11, cube_SGS_11 = get_fields(filter_width_n=5,origin_x=0,sample_nx=100,field="u",t=0)
cube_grad_12, cube_SGS_12 = get_fields(filter_width_n=5,origin_x=np.pi,sample_nx=100,field="u",t=0)

In [ ]:
np.save('cube_11_grad', cube_grad_11)
np.save('cube_12_grad', cube_grad_12)

np.save('cube_11_SGS', cube_SGS_11)
np.save('cube_12_SGS', cube_SGS_12)

In [ ]:
cube_grad_9, cube_SGS_9 = get_fields(filter_width_n=9,origin_x=0,sample_nx=100,field="u",t=0,dataset = "isotropic4096")
cube_grad_10, cube_SGS_10 = get_fields(filter_width_n=9,origin_x=0,sample_nx=100,field="u",t=0,dataset = "isotropic4096")


np.save('cube_9_grad', cube_grad)
np.save('cube_9_SGS', cube_SGS)

np.save('cube_10_grad', cube_grad)
np.save('cube_10_SGS', cube_SGS)

In [ ]:
np.save('cube_9_grad', cube_grad_9)
np.save('cube_9_SGS', cube_SGS_9)

np.save('cube_10_grad', cube_grad_10)
np.save('cube_10_SGS', cube_SGS_10)

print(np.shape(cube_grad_9))
print(np.shape(cube_SGS_9))

print(np.shape(cube_grad_10))
print(np.shape(cube_SGS_10))